In [ ]:
pip install dash plotly requests pandas numpy

In [ ]:
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import requests
from datetime import datetime, timedelta
import numpy as np

# Initialize Dash app
app = dash.Dash(__name__)

# Define the base URL for the NY Fed API
BASE_URL = "https://markets.newyorkfed.org/api"

# Define API endpoints based on the images
ENDPOINTS = {
    'ambs': {
        'current': '/ambs/{operation}/{status}/{include}/latest.{format}',
        'two_weeks': '/ambs/{operation}/results/{include}/lastTwoWeeks.{format}',
        'last_n': '/ambs/{operation}/results/{include}/last/{number}.{format}',
        'search': '/ambs/{operation}/results/{include}/search.{format}'
    },
    'fxs': {
        'current': '/fxs/{operationType}/latest.{format}',
        'last_n': '/fxs/{operationType}/last/{number}.{format}',
        'search': '/fxs/{operationType}/search.{format}',
        'counterparties': '/fxs/list/counterparties.{format}'
    },
    'guidesheets': {
        'latest': '/guidesheets/{guidesheetType}/latest.{format}',
        'previous': '/guidesheets/{guidesheetType}/previous.{format}'
    },
    'pd': {
        'latest_break': '/pd/latest/{seriesbreak}.{format}',
        'all_timeseries': '/pd/get/all/timeseries.csv',
        'timeseries': '/pd/list/timeseries.{format}',
        'asof': '/pd/list/asof.{format}',
        'seriesbreaks': '/pd/list/seriesbreaks.{format}',
        'by_date': '/pd/get/asof/{date}.{format}',
        'by_timeseries': '/pd/get/{timeseries}.{format}',
        'by_break_series': '/pd/get/{seriesbreak}/timeseries/{timeseries}.{format}'
    },
    'marketshare': {
        'quarterly': '/marketshare/qtrly/latest.{format}',
        'ytd': '/marketshare/ytd/latest.{format}'
    },
    'rates': {
        'all_latest': '/rates/all/latest.{format}',
        'search': '/rates/all/search.{format}',
        'secured_latest': '/rates/secured/all/latest.{format}',
        'secured_last': '/rates/secured/{ratetype}/last/{number}.{format}',
        'secured_search': '/rates/secured/{ratetype}/search.{format}',
        'unsecured_latest': '/rates/unsecured/all/latest.{format}',
        'unsecured_last': '/rates/unsecured/{ratetype}/last/{number}.{format}',
        'unsecured_search': '/rates/unsecured/{ratetype}/search.{format}'
    },
    'rp': {
        'current': '/rp/{operationType}/{method}/{status}/latest.{format}',
        'two_weeks': '/rp/{operationType}/{method}/results/lastTwoWeeks.{format}',
        'last_n': '/rp/{operationType}/{method}/results/last/{number}.{format}',
        'search': '/rp/results/search.{format}',
        'propositions': '/rp/reverserepo/propositions/search.{format}'
    },
    'seclending': {
        'current': '/seclending/{operation}/results/{include}/latest.{format}',
        'two_weeks': '/seclending/{operation}/results/{include}/lastTwoWeeks.{format}',
        'last_n': '/seclending/{operation}/results/{include}/last/{number}.{format}',
        'search': '/seclending/{operation}/results/{include}/search.{format}'
    },
    'tsy': {
        'current': '/tsy/{operation}/{status}/{include}/latest.{format}',
        'two_weeks': '/tsy/{operation}/results/{include}/lastTwoWeeks.{format}',
        'last_n': '/tsy/{operation}/results/{include}/last/{number}.{format}',
        'search': '/tsy/{operation}/results/{include}/search.{format}'
    },
    'soma': {
        'latest': '/soma/asofdates/latest.{format}',
        'summary': '/soma/summary.{format}',
        'dates_list': '/soma/asofdates/list.{format}',
        'agency_release': '/soma/agency/get/release_log.{format}',
        'agency_by_date': '/soma/agency/get/asof/{date}.{format}',
        'agency_by_cusip': '/soma/agency/get/cusip/{cusip}.{format}',
        'agency_by_holding': '/soma/agency/get/{holdingtype}/asof/{date}.{format}',
        'agency_wam': '/soma/agency/wam/agency_debts/asof/{date}.{format}',
        'tsy_release': '/soma/tsy/get/release_log.{format}',
        'tsy_by_date': '/soma/tsy/get/asof/{date}.{format}',
        'tsy_by_cusip': '/soma/tsy/get/cusip/{cusip}.{format}',
        'tsy_by_holding': '/soma/tsy/get/{holdingtype}/asof/{date}.{format}',
        'tsy_wam': '/soma/tsy/wam/{holdingtype}/asof/{date}.{format}',
        'tsy_monthly': '/soma/tsy/get/monthly.{format}'
    }
}

# Helper function to fetch data from API
def fetch_data(endpoint, params=None):
    try:
        response = requests.get(BASE_URL + endpoint, params=params)
        if response.status_code == 200:
            return response.json()
        else:
            return None
    except:
        return None

# Create sample data for demonstration
def create_sample_data():
    dates = pd.date_range(end=datetime.now(), periods=100, freq='D')

    # Sample SOMA holdings data
    soma_data = pd.DataFrame({
        'date': dates,
        'treasury_holdings': np.random.normal(5000, 200, 100).cumsum() + 5000000,
        'agency_holdings': np.random.normal(2000, 100, 100).cumsum() + 2000000,
        'mbs_holdings': np.random.normal(3000, 150, 100).cumsum() + 3000000
    })

    # Sample reference rates data
    rates_data = pd.DataFrame({
        'date': dates,
        'SOFR': np.random.normal(5.25, 0.1, 100),
        'EFFR': np.random.normal(5.33, 0.05, 100),
        'OBFR': np.random.normal(5.32, 0.05, 100),
        'TGCR': np.random.normal(5.30, 0.08, 100),
        'BGCR': np.random.normal(5.28, 0.08, 100)
    })

    # Sample repo operations data
    repo_data = pd.DataFrame({
        'date': dates,
        'repo_volume': np.random.normal(500, 50, 100).cumsum() + 50000,
        'reverse_repo_volume': np.random.normal(2000, 200, 100).cumsum() + 200000
    })

    return soma_data, rates_data, repo_data

# Create the layout
app.layout = html.Div([
    html.Div([
        html.H1("Markets Data Dashboard", style={'textAlign': 'center', 'marginBottom': '30px'}),
        html.P("New York Fed Market Operations", style={'textAlign': 'center', 'fontSize': '18px', 'color': '#666'})
    ], style={'backgroundColor': '#f0f0f0', 'padding': '20px', 'marginBottom': '30px'}),

    # Navigation tabs
    dcc.Tabs(id='main-tabs', value='soma', children=[
        dcc.Tab(label='SOMA Holdings', value='soma'),
        dcc.Tab(label='Reference Rates', value='rates'),
        dcc.Tab(label='Repo Operations', value='repo'),
        dcc.Tab(label='Securities Lending', value='seclending'),
        dcc.Tab(label='Primary Dealer Stats', value='pd'),
        dcc.Tab(label='Agency MBS', value='ambs')
    ], style={'marginBottom': '20px'}),

    # Content area
    html.Div(id='tab-content', style={'padding': '20px'}),

    # Interval component for auto-refresh
    dcc.Interval(
        id='interval-component',
        interval=60*1000,  # Update every minute
        n_intervals=0
    )
])

# Callback for tab content
@app.callback(
    Output('tab-content', 'children'),
    [Input('main-tabs', 'value')]
)
def render_content(tab):
    soma_data, rates_data, repo_data = create_sample_data()

    if tab == 'soma':
        return html.Div([
            html.H2("System Open Market Account Holdings"),
            html.P("SOMA Holdings (Please take note that Agency includes Agency Debts, MBS, and CMBS. Treasury includes Bills, Notes/Bonds, FRN, and TIPS.)"),

            dcc.Graph(
                figure={
                    'data': [
                        go.Scatter(x=soma_data['date'], y=soma_data['treasury_holdings'],
                                 name='Treasury Securities', line=dict(color='#1f77b4', width=2)),
                        go.Scatter(x=soma_data['date'], y=soma_data['agency_holdings'],
                                 name='Agency Securities', line=dict(color='#ff7f0e', width=2)),
                        go.Scatter(x=soma_data['date'], y=soma_data['mbs_holdings'],
                                 name='MBS Holdings', line=dict(color='#2ca02c', width=2))
                    ],
                    'layout': go.Layout(
                        title='SOMA Portfolio Composition',
                        xaxis={'title': 'Date'},
                        yaxis={'title': 'Holdings ($ Millions)'},
                        hovermode='x unified',
                        showlegend=True,
                        height=500
                    )
                }
            ),

            html.Div([
                html.H3("Summary Statistics"),
                html.Table([
                    html.Tr([
                        html.Th("Security Type"),
                        html.Th("Current Holdings"),
                        html.Th("Change (1 Week)"),
                        html.Th("Change (1 Month)")
                    ]),
                    html.Tr([
                        html.Td("Treasury Securities"),
                        html.Td(f"${soma_data['treasury_holdings'].iloc[-1]:,.0f}M"),
                        html.Td(f"{(soma_data['treasury_holdings'].iloc[-1] - soma_data['treasury_holdings'].iloc[-7])/soma_data['treasury_holdings'].iloc[-7]*100:.2f}%"),
                        html.Td(f"{(soma_data['treasury_holdings'].iloc[-1] - soma_data['treasury_holdings'].iloc[-30])/soma_data['treasury_holdings'].iloc[-30]*100:.2f}%")
                    ]),
                    html.Tr([
                        html.Td("Agency Securities"),
                        html.Td(f"${soma_data['agency_holdings'].iloc[-1]:,.0f}M"),
                        html.Td(f"{(soma_data['agency_holdings'].iloc[-1] - soma_data['agency_holdings'].iloc[-7])/soma_data['agency_holdings'].iloc[-7]*100:.2f}%"),
                        html.Td(f"{(soma_data['agency_holdings'].iloc[-1] - soma_data['agency_holdings'].iloc[-30])/soma_data['agency_holdings'].iloc[-30]*100:.2f}%")
                    ])
                ], style={'margin': '20px auto', 'width': '80%'})
            ])
        ])

    elif tab == 'rates':
        return html.Div([
            html.H2("Reference Rates"),
            html.P("Secured and Unsecured Rates"),

            dcc.Graph(
                figure={
                    'data': [
                        go.Scatter(x=rates_data['date'], y=rates_data['SOFR'],
                                 name='SOFR', line=dict(color='#1f77b4', width=2)),
                        go.Scatter(x=rates_data['date'], y=rates_data['EFFR'],
                                 name='EFFR', line=dict(color='#ff7f0e', width=2)),
                        go.Scatter(x=rates_data['date'], y=rates_data['OBFR'],
                                 name='OBFR', line=dict(color='#2ca02c', width=2)),
                        go.Scatter(x=rates_data['date'], y=rates_data['TGCR'],
                                 name='TGCR', line=dict(color='#d62728', width=2)),
                        go.Scatter(x=rates_data['date'], y=rates_data['BGCR'],
                                 name='BGCR', line=dict(color='#9467bd', width=2))
                    ],
                    'layout': go.Layout(
                        title='Reference Rates Over Time',
                        xaxis={'title': 'Date'},
                        yaxis={'title': 'Rate (%)'},
                        hovermode='x unified',
                        showlegend=True,
                        height=500
                    )
                }
            ),

            html.Div([
                html.H3("Current Rates"),
                html.Table([
                    html.Tr([
                        html.Th("Rate"),
                        html.Th("Current"),
                        html.Th("Previous"),
                        html.Th("Change")
                    ]),
                    html.Tr([
                        html.Td("SOFR"),
                        html.Td(f"{rates_data['SOFR'].iloc[-1]:.3f}%"),
                        html.Td(f"{rates_data['SOFR'].iloc[-2]:.3f}%"),
                        html.Td(f"{rates_data['SOFR'].iloc[-1] - rates_data['SOFR'].iloc[-2]:.3f}%")
                    ]),
                    html.Tr([
                        html.Td("EFFR"),
                        html.Td(f"{rates_data['EFFR'].iloc[-1]:.3f}%"),
                        html.Td(f"{rates_data['EFFR'].iloc[-2]:.3f}%"),
                        html.Td(f"{rates_data['EFFR'].iloc[-1] - rates_data['EFFR'].iloc[-2]:.3f}%")
                    ])
                ], style={'margin': '20px auto', 'width': '60%'})
            ])
        ])

    elif tab == 'repo':
        return html.Div([
            html.H2("Repo Operations"),
            html.P("Overnight Repo and Reverse Repo Volumes"),

            dcc.Graph(
                figure={
                    'data': [
                        go.Scatter(x=repo_data['date'], y=repo_data['repo_volume'],
                                 name='Repo Volume', line=dict(color='#1f77b4', width=2)),
                        go.Scatter(x=repo_data['date'], y=repo_data['reverse_repo_volume'],
                                 name='Reverse Repo Volume', line=dict(color='#ff7f0e', width=2))
                    ],
                    'layout': go.Layout(
                        title='Repo Operations Volume Over Time',
                        xaxis={'title': 'Date'},
                        yaxis={'title': 'Volume ($ Millions)'},
                        hovermode='x unified',
                        showlegend=True,
                        height=500
                    )
                }
            ),
            html.Div([
                html.H3("Summary Statistics"),
                html.Table([
                    html.Tr([
                        html.Th("Operation"),
                        html.Th("Current Volume"),
                        html.Th("Change (1 Week)"),
                        html.Th("Change (1 Month)")
                    ]),
                    html.Tr([
                        html.Td("Repo Volume"),
                        html.Td(f"${repo_data['repo_volume'].iloc[-1]:,.0f}M"),
                        html.Td(f"{(repo_data['repo_volume'].iloc[-1] - repo_data['repo_volume'].iloc[-7])/repo_data['repo_volume'].iloc[-7]*100:.2f}%"),
                        html.Td(f"{(repo_data['repo_volume'].iloc[-1] - repo_data['repo_volume'].iloc[-30])/repo_data['repo_volume'].iloc[-30]*100:.2f}%")
                    ]),
                    html.Tr([
                        html.Td("Reverse Repo Volume"),
                        html.Td(f"${repo_data['reverse_repo_volume'].iloc[-1]:,.0f}M"),
                        html.Td(f"{(repo_data['reverse_repo_volume'].iloc[-1] - repo_data['reverse_repo_volume'].iloc[-7])/repo_data['reverse_repo_volume'].iloc[-7]*100:.2f}%"),
                        html.Td(f"{(repo_data['reverse_repo_volume'].iloc[-1] - repo_data['reverse_repo_volume'].iloc[-30])/repo_data['reverse_repo_volume'].iloc[-30]*100:.2f}%")
                    ])
                ], style={'margin': '20px auto', 'width': '80%'})
            ])
        ])

In [ ]:
# Test the dashboard code by running it
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Create sample data for testing
dates = pd.date_range(end=datetime.now(), periods=365, freq='D')

# SOMA Holdings data
soma_data = pd.DataFrame({
    'date': dates,
    'treasury_holdings': np.random.normal(5500, 100, 365).cumsum() + 5000000,
    'agency_holdings': np.random.normal(2500, 50, 365).cumsum() + 2500000
})

# Rates data
rates_data = pd.DataFrame({
    'date': dates,
    'SOFR': np.random.normal(5.25, 0.1, 365),
    'EFFR': np.random.normal(5.33, 0.1, 365),
    'OBFR': np.random.normal(5.32, 0.1, 365),
    'TGCR': np.random.normal(5.24, 0.1, 365),
    'BGCR': np.random.normal(5.23, 0.1, 365)
})

# Repo operations data
repo_data = pd.DataFrame({
    'date': dates,
    'overnight_repo': np.random.normal(500, 50, 365),
    'term_repo': np.random.normal(100, 20, 365),
    'overnight_rrp': np.random.normal(2000, 200, 365)
})

# Primary dealer data
pd_data = pd.DataFrame({
    'date': dates[-52:],  # Weekly data
    'treasury_positions': np.random.normal(200, 30, 52),
    'agency_positions': np.random.normal(50, 10, 52),
    'mbs_positions': np.random.normal(150, 25, 52)
})

print("Sample data created successfully")
print(f"SOMA data shape: {soma_data.shape}")
print(f"Rates data shape: {rates_data.shape}")
print(f"Repo data shape: {repo_data.shape}")
print(f"Primary dealer data shape: {pd_data.shape}")

In [ ]:
# Create a complete NY Fed Markets Data Dashboard
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Initialize the Dash app
app = dash.Dash(__name__)

# Create sample data
dates = pd.date_range(end=datetime.now(), periods=365, freq='D')

# SOMA Holdings data
soma_data = pd.DataFrame({
    'date': dates,
    'treasury_holdings': np.random.normal(5500, 100, 365).cumsum() + 5000000,
    'agency_holdings': np.random.normal(2500, 50, 365).cumsum() + 2500000
})

# Rates data
rates_data = pd.DataFrame({
    'date': dates,
    'SOFR': np.random.normal(5.25, 0.1, 365),
    'EFFR': np.random.normal(5.33, 0.1, 365),
    'OBFR': np.random.normal(5.32, 0.1, 365),
    'TGCR': np.random.normal(5.24, 0.1, 365),
    'BGCR': np.random.normal(5.23, 0.1, 365)
})

# Repo operations data
repo_data = pd.DataFrame({
    'date': dates,
    'overnight_repo': np.random.normal(500, 50, 365),
    'term_repo': np.random.normal(100, 20, 365),
    'overnight_rrp': np.random.normal(2000, 200, 365)
})

# Primary dealer data
pd_data = pd.DataFrame({
    'date': dates[-52:],  # Weekly data
    'treasury_positions': np.random.normal(200, 30, 52),
    'agency_positions': np.random.normal(50, 10, 52),
    'mbs_positions': np.random.normal(150, 25, 52)
})

# Define the app layout
app.layout = html.Div([
    html.Div([
        html.H1("New York Fed Markets Data", style={'textAlign': 'center', 'color': '#003366'}),
        html.P("Market Operations and Reference Rates Dashboard",
               style={'textAlign': 'center', 'fontSize': '18px', 'color': '#666'})
    ], style={'backgroundColor': '#f0f0f0', 'padding': '20px', 'marginBottom': '20px'}),

    dcc.Tabs(id='tabs', value='soma', children=[
        dcc.Tab(label='SOMA Holdings', value='soma'),
        dcc.Tab(label='Reference Rates', value='rates'),
        dcc.Tab(label='Repo Operations', value='repo'),
        dcc.Tab(label='Primary Dealers', value='pd'),
        dcc.Tab(label='API Documentation', value='api')
    ]),

    html.Div(id='tab-content', style={'padding': '20px'})
])

# Callback for tab content
@app.callback(Output('tab-content', 'children'),
              Input('tabs', 'value'))
def render_content(tab):
    if tab == 'soma':
        return html.Div([
            html.H2("System Open Market Account (SOMA) Holdings"),
            html.P("Federal Reserve Securities Holdings"),

            dcc.Graph(
                figure={
                    'data': [
                        go.Scatter(x=soma_data['date'], y=soma_data['treasury_holdings']/1e6,
                                 name='Treasury Securities', line=dict(color='#1f77b4', width=2)),
                        go.Scatter(x=soma_data['date'], y=soma_data['agency_holdings']/1e6,
                                 name='Agency Securities', line=dict(color='#ff7f0e', width=2))
                    ],
                    'layout': go.Layout(
                        title='SOMA Portfolio Holdings (Trillions USD)',
                        xaxis={'title': 'Date'},
                        yaxis={'title': 'Holdings ($ Trillions)'},
                        hovermode='x unified',
                        showlegend=True,
                        height=500
                    )
                }
            ),

            html.Div([
                html.H3("Current Holdings Summary"),
                html.Table([
                    html.Tr([
                        html.Th("Security Type"),
                        html.Th("Current Holdings"),
                        html.Th("Week Change"),
                        html.Th("Month Change")
                    ]),
                    html.Tr([
                        html.Td("Treasury Securities"),
                        html.Td(f"${soma_data['treasury_holdings'].iloc[-1]/1e6:.2f}T"),
                        html.Td(f"{(soma_data['treasury_holdings'].iloc[-1] - soma_data['treasury_holdings'].iloc[-7])/soma_data['treasury_holdings'].iloc[-7]*100:.2f}%"),
                        html.Td(f"{(soma_data['treasury_holdings'].iloc[-1] - soma_data['treasury_holdings'].iloc[-30])/soma_data['treasury_holdings'].iloc[-30]*100:.2f}%")
                    ]),
                    html.Tr([
                        html.Td("Agency Securities"),
                        html.Td(f"${soma_data['agency_holdings'].iloc[-1]/1e6:.2f}T"),
                        html.Td(f"{(soma_data['agency_holdings'].iloc[-1] - soma_data['agency_holdings'].iloc[-7])/soma_data['agency_holdings'].iloc[-7]*100:.2f}%"),
                        html.Td(f"{(soma_data['agency_holdings'].iloc[-1] - soma_data['agency_holdings'].iloc[-30])/soma_data['agency_holdings'].iloc[-30]*100:.2f}%")
                    ])
                ], style={'margin': '20px auto', 'width': '80%'})
            ])
        ])

    elif tab == 'rates':
        return html.Div([
            html.H2("Reference Rates"),
            html.P("Secured and Unsecured Rates"),

            dcc.Graph(
                figure={
                    'data': [
                        go.Scatter(x=rates_data['date'], y=rates_data['SOFR'],
                                 name='SOFR', line=dict(color='#1f77b4', width=2)),
                        go.Scatter(x=rates_data['date'], y=rates_data['EFFR'],
                                 name='EFFR', line=dict(color='#ff7f0e', width=2)),
                        go.Scatter(x=rates_data['date'], y=rates_data['OBFR'],
                                 name='OBFR', line=dict(color='#2ca02c', width=2)),
                        go.Scatter(x=rates_data['date'], y=rates_data['TGCR'],
                                 name='TGCR', line=dict(color='#d62728', width=2)),
                        go.Scatter(x=rates_data['date'], y=rates_data['BGCR'],
                                 name='BGCR', line=dict(color='#9467bd', width=2))
                    ],
                    'layout': go.Layout(
                        title='Reference Rates Over Time',
                        xaxis={'title': 'Date'},
                        yaxis={'title': 'Rate (%)'},
                        hovermode='x unified',
                        showlegend=True,
                        height=500
                    )
                }
            ),

            html.Div([
                html.H3("Current Rates"),
                html.Table([
                    html.Tr([
                        html.Th("Rate"),
                        html.Th("Current"),
                        html.Th("Previous"),
                        html.Th("Change")
                    ]),
                    html.Tr([
                        html.Td("SOFR"),
                        html.Td(f"{rates_data['SOFR'].iloc[-1]:.3f}%"),
                        html.Td(f"{rates_data['SOFR'].iloc[-2]:.3f}%"),
                        html.Td(f"{rates_data['SOFR'].iloc[-1] - rates_data['SOFR'].iloc[-2]:.3f}%")
                    ]),
                    html.Tr([
                        html.Td("EFFR"),
                        html.Td(f"{rates_data['EFFR'].iloc[-1]:.3f}%"),
                        html.Td(f"{rates_data['EFFR'].iloc[-2]:.3f}%"),
                        html.Td(f"{rates_data['EFFR'].iloc[-1] - rates_data['EFFR'].iloc[-2]:.3f}%")
                    ])
                ], style={'margin': '20px auto', 'width': '60%'})
            ])
        ])

    elif tab == 'repo':
        return html.Div([
            html.H2("Repo and Reverse Repo Operations"),
            html.P("Open Market Operations"),

            dcc.Graph(
                figure={
                    'data': [
                        go.Bar(x=repo_data['date'][-30:], y=repo_data['overnight_repo'][-30:],
                               name='Overnight Repo', marker_color='#1f77b4'),
                        go.Bar(x=repo_data['date'][-30:], y=repo_data['term_repo'][-30:],
                               name='Term Repo', marker_color='#ff7f0e'),
                        go.Bar(x=repo_data['date'][-30:], y=repo_data['overnight_rrp'][-30:],
                               name='Overnight RRP', marker_color='#2ca02c')
                    ],
                    'layout': go.Layout(
                        title='Recent Repo Operations (Last 30 Days)',
                        xaxis={'title': 'Date'},
                        yaxis={'title': 'Volume ($ Billions)'},
                        barmode='group',
                        showlegend=True,
                        height=500
                    )
                }
            )
        ])

    elif tab == 'pd':
        return html.Div([
            html.H2("Primary Dealer Statistics"),
            html.P("Weekly Dealer Positions"),

            dcc.Graph(
                figure={
                    'data': [
                        go.Scatter(x=pd_data['date'], y=pd_data['treasury_positions'],
                                 name='Treasury', line=dict(color='#1f77b4', width=2)),
                        go.Scatter(x=pd_data['date'], y=pd_data['agency_positions'],
                                 name='Agency', line=dict(color='#ff7f0e', width=2)),
                        go.Scatter(x=pd_data['date'], y=pd_data['mbs_positions'],
                                 name='MBS', line=dict(color='#2ca02c', width=2))
                    ],
                    'layout': go.Layout(
                        title='Primary Dealer Net Positions',
                        xaxis={'title': 'Date'},
                        yaxis={'title': 'Position ($ Billions)'},
                        hovermode='x unified',
                        showlegend=True,
                        height=500
                    )
                }
            )
        ])

    elif tab == 'api':
        return html.Div([
            html.H2("Markets Data API"),
            html.P("Access New York Fed market data programmatically"),

            html.Div([
                html.H3("Available Endpoints"),
                html.Div([
                    html.H4("SOMA Holdings"),
                    html.Pre("""
GET /api/soma/summary/latest.json
GET /api/soma/details/search.json?startDate={date}&endDate={date}
                    """, style={'backgroundColor': '#f5f5f5', 'padding': '10px'}),

                    html.H4("Reference Rates"),
                    html.Pre("""
GET /api/rates/all/latest.json
GET /api/rates/secured/all/latest.json
GET /api/rates/unsecured/all/latest.json
                    """, style={'backgroundColor': '#f5f5f5', 'padding': '10px'}),

                    html.H4("Repo Operations"),
                    html.Pre("""
GET /api/rp/repo/all/results/latest.json
GET /api/rp/reverserepo/all/results/latest.json
GET /api/rp/all/all/results/lastTwoWeeks.json
                    """, style={'backgroundColor': '#f5f5f5', 'padding': '10px'}),

                    html.H4("Primary Dealer Statistics"),
                    html.Pre("""
GET /api/pd/latest/all.json
GET /api/pd/get/all/timeseries.csv
                    """, style={'backgroundColor': '#f5f5f5', 'padding': '10px'})
                ])
            ], style={'marginTop': '20px'})
        ])

    return html.Div()

print("Dashboard created successfully!")
print("To run the dashboard, execute: app.run_server(debug=True)")

In [ ]:
app.run_server(debug=True, mode='inline')